# Init

In [4]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder \
     .appName("Test") \
     .getOrCreate()

In [5]:
import pandas as pd
pd.options.display.max_rows=250
import numpy as np
from datetime import datetime

import matplotlib.pyplot as plt
plt.style.use('seaborn')
import matplotlib.ticker as ticker
from matplotlib.ticker import FormatStrFormatter, FuncFormatter

import pyspark.sql.functions as func

# Dataset Imports

In [6]:
start = datetime.now()

df_mentions = spark.read.parquet("s3://labadie-gdelt-tradewar/mentions.parquet")
df_mentions.cache()

print(df_mentions.count())
print(df_mentions.printSchema())
print(datetime.now()-start)

329962831
root
 |-- EventDate: string (nullable = true)
 |-- MentionSource: string (nullable = true)
 |-- MentionIdentifier: string (nullable = true)
 |-- MentionDocTone: string (nullable = true)

None
0:02:20.214583


# Model

### Create Features

In [26]:
start = datetime.now()

print(df_mentions.select(["MentionSource"]).distinct().count())
print()
print(df_mentions.groupby("MentionSource").count().orderBy("count", ascending=False).show())
print()

print(datetime.now()-start)

110284

+--------------------+-------+
|       MentionSource|  count|
+--------------------+-------+
|          iheart.com|5415499|
|           yahoo.com|3544136|
|         reuters.com|2101841|
|              ap.org|1872987|
|     dailymail.co.uk|1735160|
|business-standard...|1604209|
|      indiatimes.com|1470522|
|     english.wafa.ps|1363294|
|          sfgate.com|1117552|
|             msn.com|1083702|
|  washingtonpost.com|1012533|
|           chron.com| 923878|
|        philstar.com| 899644|
|       4-traders.com| 899315|
|       allafrica.com| 656913|
|          tucson.com| 648131|
|  couriermail.com.au| 640124|
|     wickedlocal.com| 594929|
|         foxnews.com| 589656|
|    heraldsun.com.au| 581688|
+--------------------+-------+
only showing top 20 rows

None

0:00:14.265990
